In [6]:
import time
import mxnet as mx
from mxnet.gluon import nn, Trainer, loss
from mxnet import optimizer
from mxnet import autograd, np, npx, gpu
npx.set_np()
mx.context.num_gpus()
from numba import cuda
cuda.select_device(0)

<weakproxy at 0x7f2c746db458 to Device at 0x7f2c8b8fb198>

In [9]:
gpu = mx.gpu()

In [10]:
gpu

gpu(0)

In [11]:
def get_model():
    from tensorflow.keras.models import Sequential
    from tensorflow.keras import initializers
    from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten, BatchNormalization
    model = Sequential()

    model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(320, 320, 1), activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(320, 320, 1), activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(Dropout(0.2))

    model.add(Dense(1, activation='sigmoid',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [12]:

import os
def fullpath(path, files):
    return  [(lambda x: path + x)(x) for x in files]
val_dir = "../../chest_xray/chest_xray/val"
filesPneumo = fullpath(val_dir+'/PNEUMONIA/',os.listdir(os.path.join(val_dir, 'PNEUMONIA')))
del filesPneumo[0]
labelsPneumo = np.zeros(len(filesPneumo))
filesNormal = fullpath(val_dir+'/NORMAL/',os.listdir(os.path.join(val_dir, 'NORMAL')))
del filesNormal[0]
labelsNormal = np.ones(len(filesNormal))
fileData = filesPneumo + filesNormal
fileLabels = np.concatenate((labelsPneumo,labelsNormal),axis=0).astype(np.float32)

In [13]:

model = get_model()


In [14]:
from tensorflow.keras.backend import clear_session
def convert_keras_to_mxnet(model):
    def layer_conversion(layer):
        t = str(type(layer)).split(".")
        layer_type = t[-1].replace('\'>',"")
        config = layer.get_config()
        if layer_type == "Dense":
            return nn.Dense(units=config['units'], activation=config['activation'], use_bias=config['use_bias'])
        if layer_type == "Conv2D":
            return nn.Conv2D(channels=config['filters'], kernel_size=config['kernel_size'],use_bias=config['use_bias'],activation=config['activation'], weight_initializer=mx.init.Normal(0.01), bias_initializer=mx.init.Zero())
        if layer_type == "MaxPooling2D":
            return nn.MaxPool2D(pool_size=config['pool_size'])
        if layer_type == "BatchNormalization":
            return nn.BatchNorm(momentum=config['momentum'], epsilon=config['epsilon'])
        if layer_type=="Dropout":
            return nn.Dropout(rate=config['rate'])
        if layer_type=="Flatten":
            return nn.Flatten()
        return None
    mxModel = nn.Sequential()
    for layer in model.layers:
        new_layer = layer_conversion(layer)
        mxModel.add(new_layer)
    mxModel.initialize(force_reinit=True, init=mx.init.Xavier())
    return mxModel

new_model = convert_keras_to_mxnet(model)
clear_session()

cuda.select_device(0)
cuda.close()
del model

In [15]:
new_model.initialize(force_reinit=True, init=mx.init.Xavier())

In [16]:
cuda.select_device(0)

<weakproxy at 0x7f2c746db458 to Device at 0x7f2c8b8fb198>

In [ ]:

new_model.load_parameters('new_model.params', ctx=gpu)

NameError: name 'new_model' is not defined

In [8]:
from mxnet import image
def preprocess(img,label):
    img = image.imresize(img/ 255.0,320,320).astype(np.float32)
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)
    #img = img[np.newaxis, :].astype(np.float32)
    return img,label
ds2 =  mx.gluon.data.vision.datasets.ImageFolderDataset("../../chest_xray/chest_xray/val",transform=preprocess)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Ignoring ../../chest_xray/chest_xray/val/.DS_Store, which is not a directory.
  
/usr/local/lib/python3.6/dist-packages/mxnet/gluon/data/vision/datasets.py:318: UserWarning: Ignoring ../../chest_xray/chest_xray/val/NORMAL/.DS_Store of type . Only support .jpg, .jpeg, .png
  filename, ext, ', '.join(self._exts)))
/usr/local/lib/python3.6/dist-packages/mxnet/gluon/data/vision/datasets.py:318: UserWarning: Ignoring ../../chest_xray/chest_xray/val/PNEUMONIA/.DS_Store of type . Only support .jpg, .jpeg, .png
  filename, ext, ', '.join(self._exts)))


In [9]:
train_data = mx.gluon.data.DataLoader(ds2, batch_size=16, shuffle=True, num_workers=4)


In [9]:
def get_batches(dataset, batch_size, labels):
    num_steps = len(dataset)//batch_size
    steps = range(num_steps)
    data_batches = []
    label_batches = []
    for step in steps:
        batch_idx = range(batch_size) if len(dataset.items[step*batch_size:])>batch_size else range(len(dataset.items[step*batch_size:]))
        for idx in batch_idx:
            if idx == 0:
                img_arr = dataset[step*batch_size+idx][0]

            else:
                img_arr = np.concatenate((img_arr,dataset[step*batch_size+idx][0]),axis=0)
        label_batches.append(labels[step*batch_size:step*batch_size+len(batch_idx)])
        data_batches.append(img_arr)
    return  data_batches, label_batches

In [9]:
data_batches, label_batches = get_batches(ds2, 16, fileLabels)

In [24]:
y_true = np.array([0.,0.,1.,1.]).reshape(-1,1)

In [ ]:
params = new_model[-1].collect_params()

In [ ]:
dense_W = params.get("weight").data()
dense_B = params.get("bias").data()

In [ ]:
dense_W.attach_grad()
dense_B.attach_grad()


In [ ]:
dense_W

In [27]:
trainer=mx.gluon.Trainer(new_model.collect_params(),'adam', {'learning_rate': 0.001})

In [25]:
images = ds2[0][0]
images = np.concatenate((images,ds2[1][0], ds2[-1][0], ds2[-2][0]), axis=0)

In [46]:
with autograd.record():
    y_ =  new_model.forward(images)
    lossValue = BCE(y_, y_true)
y_[y_<0.5]=0
y_[y_>=0.5]=1
acc = (sum(y_==y_true)/len(y_true))*100
print( "acc: " + str(acc) + "; Loss: " + str(lossValue))

acc: [100.]; Loss: [0.69314724 0.69314724 0.31326175 0.31326175]


In [10]:
def acc(output, label):
    # output: (batch, num_output) float32 ndarray
    # label: (batch, ) int32 ndarray
    pred = np.zeros(len(output))
    print("Predictions: " + str(output) + ";  Ground Truth: " + str(label))
    for i,o in enumerate(output):
        pred[i] = 0. if o < 0.5 else 1.
    return (pred ==
            label.astype('float32')).mean()

In [11]:
def train(dataset_batches, labels, epochs, learning_rate):
    train_loss, train_acc = 0., 0.
    trainerTEST = mx.gluon.Trainer(new_model.collect_params(),'adam', {'learning_rate': learning_rate})
    BCE = loss.SigmoidBinaryCrossEntropyLoss()
    epochs = range(epochs)
    for epoch in epochs:
        for data_batch, labels_batch in zip(dataset_batches, labels):
            y__true = labels_batch.reshape(-1,1)
            with autograd.record():
                y__ =  new_model(data_batch)
                lossValueTEST = BCE(y__, y__true)
            lossValueTEST.backward()
            trainerTEST.step(16)
        train_loss = lossValueTEST.mean()
        train_acc = acc(y__, y__true)
        print( "Epoch: "+ str(epoch+1) +"; acc: " + str(train_acc)+"; Loss: " + str(train_loss))

In [12]:
train(data_batches,label_batches, 10, 0.0005)

Epoch: 1; acc: 0.5; Loss: 0.73598045
Epoch: 2; acc: 0.5; Loss: 0.6931472
Epoch: 3; acc: 0.5; Loss: 0.6931472
Epoch: 4; acc: 0.5; Loss: 0.6931472


KeyboardInterrupt: 

In [15]:
bl = label_batches[-1]

In [16]:
bl2 = bl.reshape(-1,1)

In [ ]:
del data_batches, label_batches

In [ ]:

HL = mx.gluon.loss.SigmoidBinaryCrossEntropyLoss()
trainer = mx.gluon.Trainer(new_model.collect_params(),'sgd', {'learning_rate': 0.0005})
for epoch in range(30):
    train_loss, train_acc, valid_acc = 0., 0., 0.
    tic = time.time()
    for data, label in train_data:
            # forward + backward
        with autograd.record():
            output = new_model(data)
            loss = HL(output.astype(np.float64), label.astype(np.float64))
        loss.backward()
            # update parameters
        trainer.step(16)
            # calculate training metrics
        train_loss += loss.mean()
        train_acc += acc(output, label)
        # calculate validation accuracy
    print("Epoch %d: loss %f, train acc %.3f, in %.1f sec" % (
                epoch, train_loss/len(train_data), train_acc/len(train_data), time.time()-tic))